编写Python将对全体数据进行标准化，并将数据p的特征进行重构

In [1]:
import pandas as pd
import numpy as np
import warnings
import pickle
warnings.filterwarnings('ignore')

tmp = pd.read_csv("data/p.csv")
t0_min = tmp.apply(lambda x: np.min(x), axis=0).values
t0_ptp = tmp.apply(lambda x: np.ptp(x), axis=0).values
t0 = tmp.apply(lambda x: (x - np.min(x)) / np.ptp(x), axis=0).values

SEQLEN = 14
dim_in = 108
dim_out = 96
pred_len = 30
X_train = np.zeros((t0.shape[0]-SEQLEN-pred_len, SEQLEN, dim_in))
Y_train = np.zeros((t0.shape[0]-SEQLEN-pred_len, dim_out),)
X_test = np.zeros((pred_len, SEQLEN, dim_in))
Y_test = np.zeros((pred_len, dim_out),)
for i in range(SEQLEN, t0.shape[0]-pred_len):
    Y_train[i-SEQLEN] = t0[i][0:96]
    X_train[i-SEQLEN] = np.c_[t0[(i-SEQLEN):i],t0[i+1][96:].repeat(SEQLEN).reshape((6,SEQLEN)).T]
for i in range(t0.shape[0]-pred_len, t0.shape[0]):
    Y_test[i-t0.shape[0]+pred_len] = t0[i][0:96]
    if i == t0.shape[0]-1:
        # 这里weekday、trend、month和气温数据做了近似处理，正式使用时，需要使用天气预报的数据
        X_test[i-t0.shape[0]+pred_len] = np.c_[t0[(i-SEQLEN):i],t0[i][96:].repeat(SEQLEN).reshape((6,SEQLEN)).T]
    else:
        X_test[i-t0.shape[0]+pred_len] = np.c_[t0[(i-SEQLEN):i],t0[i+1][96:].repeat(SEQLEN).reshape((6,SEQLEN)).T]

with open('data/X_train.pkl', 'wb') as f:
    pickle.dump(X_train, f)

with open('data/Y_train.pkl', 'wb') as f:
    pickle.dump(Y_train, f)

with open('data/X_test.pkl', 'wb') as f:
    pickle.dump(X_test, f)
    
with open('data/Y_test.pkl', 'wb') as f:
    pickle.dump(Y_test, f)
    
with open('data/t0_ptp.pkl', 'wb') as f:
    pickle.dump(t0_ptp, f)
    
with open('data/t0_min.pkl', 'wb') as f:
    pickle.dump(t0_min, f)